In [3]:
import mypower as myp

m = myp.start_matpower()
mypc = m.runpf(nout=1) # nout specify number of returned variable from Octave

ModuleNotFoundError: No module named 'mypower'

In [1]:
%%time
from tqdm import tqdm
import pandapower as pp
import pandas as pd
import numpy as np
import pandapower.converter as pc
from pandapower.pypower.makePTDF import makePTDF
import pyomo.environ as pyo
# import matplotlib.pyplot as plt
# import pandapower.plotting as plot

CPU times: total: 3.62 s
Wall time: 4.74 s


In [20]:
# ppc.keys()
print([ppc['branch'][0][i] for i in range(len(ppc['branch'][0]))])

[0.0, 1.0, 0.0030000000000000005, 0.014, 0.4610000000000001, 175.0, 250.0, 250.0, 1.0, 0.0, 1.0, -360.0, 360.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [12]:
%%time
mpc_file = 'data\\pglib-opf-17.08\\pglib_opf_case73_ieee_rts.m'
net = pc.from_mpc(mpc_file)
ppc = pc.to_ppc(net, init='flat')
# bus_index_dict = {}
# for i, index in enumerate(net.bus.index):
#     bus_index_dict[index] = i
# baseMVA,bus,branch = ppc['baseMVA'],ppc['bus'],ppc['branch']
# M = makePTDF(baseMVA, bus, branch)
# f_max = np.array([rateA for i in range(len(branch)) for j,rateA in enumerate(branch[i]) if j == 5])
# p_init = net.gen.p_mw.values
# p_init = np.insert(p_init, 0, 0)
# d = np.array([0 for i in range(len(net.bus))], dtype=float)
# for i, load_p_mw in enumerate(net.load.p_mw):
#     d[i] = load_p_mw
# d_shunt = np.array([0 for i in range(len(net.bus))], dtype=float)
# for i, shunt_p_mw in enumerate(net.shunt.p_mw):
#     d_shunt[i] = shunt_p_mw
# sigma_scaling = 0.03
# sigma = (sigma_scaling * d)
# mean = np.zeros(len(d))
# covariance_matrix = np.diag(sigma**2)
# num_samples = 1
# omega = np.random.multivariate_normal(mean, covariance_matrix, num_samples)
# ngen = len(net.gen) + len(net.ext_grid)
# gen_index = net.gen.bus.values
# gen_index = np.insert(gen_index, 0, net.ext_grid.bus.values)
# gen_lb = net.gen.min_p_mw.values
# gen_lb = np.insert(gen_lb, 0, np.float64(net.ext_grid.min_p_mw.values))
# gen_ub = net.gen.max_p_mw.values
# gen_ub = np.insert(gen_ub, 0, np.float64(net.ext_grid.max_p_mw.values))
# # gen_ub = np.insert(gen_ub, 0, np.float64(271.)) # 271. is from IEEE PES PGLib-OPF v17.08 only for IEEE30bus
# nbus = len(net.bus)
# nline = len(net.line)+len(net.trafo)
# cost_coeff = net.poly_cost[net.poly_cost['et'] != 'sgen'].sort_values(by='et').loc[:, 'cp0_eur': 'cp2_eur_per_mw2'].values
# connection = (net.line.from_bus.values, net.line.to_bus.values, net.trafo.hv_bus.values,\
#               net.trafo.lv_bus.values)
# bus_matrix = np.zeros( (nbus, nbus) )
# for i in zip(connection[1], connection[0]):
#     bus_matrix[bus_index_dict[i[0]],bus_index_dict[i[1]]] = 1
# gen_index_reset = np.array([bus_index_dict[gen_index[g]] for g in range(len(gen_index))])
# H = bus_matrix[:,gen_index_reset]

CPU times: total: 438 ms
Wall time: 422 ms


In [26]:
%%time
# paper model
model = pyo.ConcreteModel()
# sets
ngen = len(net.gen) + len(net.ext_grid)
model.ng = pyo.RangeSet(0, ngen-1)
nbus = len(net.bus)
model.nb = pyo.RangeSet(0, nbus-1)
nline = len(net.line) + len(net.trafo)
model.nl = pyo.RangeSet(0, nline-1)
# variables
model.p = pyo.Var(model.ng, domain=pyo.Reals, initialize=p_init)
model.omega = pyo.Var(model.nb, domain=pyo.Reals)
# objective (1a)
model.obj = pyo.Objective(expr =
                          cost_coeff[:,0].sum() + cost_coeff[:,1]@model.p + cost_coeff[:,2]@model.p
                          , sense = pyo.minimize
)
# constraint (1b) μ is omitted
model.c1b = pyo.Constraint(expr =
                          sum([model.p[g] for g in model.ng ]) == 
                           sum([d[b] + d_shunt[b] for b in model.nb]) 
                           - sum([model.omega[b] for b in model.nb])
                          )
# constraint (1c)
model.c1c_lb = pyo.ConstraintList()
model.c1c_ub = pyo.ConstraintList()
for i in model.ng:
    model.c1c_lb.add(expr=
                    gen_lb[i] <= model.p[i]
                    )
    model.c1c_ub.add(expr=
                    model.p[i] <= gen_ub[i]
                    )
# constraint (1d) μ is omitted
model.c1d_lb = pyo.ConstraintList()
model.c1d_ub = pyo.ConstraintList()
for i in model.nl:
    model.c1d_lb.add(expr=
                    -f_max[i] <= (M@(H@model.p + model.omega - (d + d_shunt)))[i]
                    )
    model.c1d_ub.add(expr=
                     (M@(H@model.p + model.omega - (d + d_shunt)))[i] <= f_max[i]
                    )
instance = model.create_instance()

ValueError: Constraint 'c1d_lb[1]' created with an invalid non-finite lower bound (nan).

In [27]:
solver = pyo.SolverFactory('mosek')
obj_dict={}
infeasible=0
for i, value in tqdm(enumerate(omega)):
    for j in model.nb:
        instance.omega[j].fix(value[j]) #
    res = solver.solve(instance)
    if not res.solver.termination_condition == 'optimal':
        infeasible+=1
#         log_infeasible_constraints(instance)
    obj_dict[i]=pyo.value(instance.obj)
print(f'C_total=', pyo.value(instance.obj))
# for i in instance.p:
#     print(f'P{i}=', pyo.value(instance.p[i]))


0it [00:00, ?it/s]


KeyError: "Index '5' is not valid for indexed component 'omega'"

In [ ]:
# from pyomo.util.infeasible import log_infeasible_constraints
# import logging
# logging.basicConfig(level=logging.INFO)
# log_infeasible_constraints(instance)